<a href="https://colab.research.google.com/github/adityarajiitmd/Deep-Learning-and-NLP/blob/main/diabetes_management_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install -q google-generativeai==0.3.1
!pip install -q gradio

In [14]:
import google.generativeai as genai
from pathlib import Path
import gradio as gr

In [15]:
genai.__version__

'0.3.1'

In [16]:
from google.colab import userdata

genai.configure(api_key = userdata.get('Gemini'))

In [17]:
generation_config = {
    "temperature": 0.4,
    "top_p": 1,
    "top_k": 32,
    "max_output_tokens": 4096,
}

safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    }
]

In [18]:
model = genai.GenerativeModel(model_name = "gemini-pro-vision",
                              generation_config = generation_config,
                              safety_settings = safety_settings)

In [29]:
def input_image_setup(file_loc):
  if not(img:= Path(file_loc)).exists():
    raise FileNotFoundError(f"Could not find image: {img}")

  image_parts = [
      {
          "mime_type": "image/jpeg",
          "data": Path(file_loc).read_bytes()
      }
  ]
  return image_parts

In [30]:
def generative_gemini_response(input_prompt, image_loc):
  image_prompt = input_image_setup(image_loc)

  prompt_parts = [input_prompt, image_prompt[0]]
  response = model.generate_content(prompt_parts)
  return response.text


In [31]:
input_prompt = """
                As an expert specializing in assessing the suitability of fruits and foods for individuals with diabetes, your task involves analyzing input images featuring various food items. Your first objective is to identify the type of fruit or food present in the image. Subsequently, you must determine the glycemic index of the identified item. Based on this glycemic index, provide recommendation on whether individuals with diabetes can include the detected food in their diet. If the food is deemed suitable, specify the recommendation quantity for consumption.
                """

In [32]:
def upload_file(files):
  file_paths = [file.name for file in files]
  if file_paths:
    response = generative_gemini_response(input_prompt, file_paths[0])
  return file_paths[0], response

with gr.Blocks() as demo:
  file_output = gr.Textbox()
  image_output = gr.Image()
  combined_output = [image_output, file_output]
  upload_button = gr.UploadButton("Click to Upload an Image of the Food you want ",
                                  file_types = ["image"],
                                  file_count="multiple")
  upload_button.upload(upload_file, upload_button, combined_output)

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://0c8831bf769152f971.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://0c8831bf769152f971.gradio.live
